# Computing fourth-order counts statistics on a mock shape catalog

In this notebooks we compute the fourth-order counts functions on a realistic catalog and compare it with the idealized prediction.

In [1]:
from astropy.table import Table
import numpy as np
from matplotlib import pyplot as plt

import orpheus

## Obtaining a realistic mock shape catalog

We will run this notebook using a mocks shape catalog from the SLICS ensemble. First, let us download this catalog and read in its contents.

**Note:** If you want to run the notebook yourself, you need to update the `savepath_SLICS` variable.

In [2]:
catname = "GalCatalog_LOS1.fits"
path_to_SLICS = "http://cuillin.roe.ac.uk/~jharno/SLICS/MockProducts/KiDS450/" + catname
savepath_SLICS = "/vol/euclidraid4/data/lporth/HigherOrderLensing/Mocks/SLICS_KiDS450/"

In [3]:
!wget {path_to_SLICS} -P {savepath_SLICS}

--2025-09-10 01:24:11--  http://cuillin.roe.ac.uk/~jharno/SLICS/MockProducts/KiDS450/GalCatalog_LOS1.fits
Resolving cuillin.roe.ac.uk (cuillin.roe.ac.uk)... 129.215.175.32
Connecting to cuillin.roe.ac.uk (cuillin.roe.ac.uk)|129.215.175.32|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196545600 (187M) [image/fits]
Saving to: ‘/vol/euclidraid4/data/lporth/HigherOrderLensing/Mocks/SLICS_KiDS450/GalCatalog_LOS1.fits.3’

GalCatalog_LOS1.fit  16%[==>                 ]  30,67M  51,1MB/s               

In [4]:
slicscat = Table.read(savepath_SLICS + catname)
print(slicscat.keys())

['x_arcmin', 'y_arcmin', 'z_spectroscopic', 'z_photometric', 'shear1', 'shear2', 'eps_obs1', 'eps_obs2']


In [5]:
fdownsample = 10

In [6]:
poscat = orpheus.ScalarTracerCatalog(
    pos1=slicscat["x_arcmin"][::fdownsample],
    pos2=slicscat["y_arcmin"][::fdownsample],
    tracer=np.ones_like(slicscat["x_arcmin"][::fdownsample]))

In [7]:
print("Number of galaxies:%i --> effective nbar: %.3f/arcmin^2 on %.2f deg^2"%(
    poscat.ngal, poscat.ngal/(poscat.len1*poscat.len2), poscat.len1*poscat.len2/3600.))

Number of galaxies:307081 --> effective nbar: 0.853/arcmin^2 on 100.00 deg^2


In [8]:
min_sep = 0.125
max_sep = 128.
binsize = 0.2
nthreads = 48
nmax=10
mapradii = np.geomspace(2,32,17)

In [9]:
fourpcf = orpheus.NNNNCorrelation_NoTomo(min_sep=min_sep, max_sep=max_sep,binsize=binsize,nmaxs=nmax,method='Tree',
                                         nthreads=nthreads,verbosity=2)

In [10]:
%%time
nap4 = fourpcf.process(poscat, lowmem=True, statistics=['4pcf_multipole','Nap4'],mapradii=mapradii)

Using batchsize of 161 for radial bins
Doing region 0/86996 for thetabatch 0/48
Computed Wn for first gal in region 0/86996 for thetabatch 0/48 in 0.0028 seconds
Allocated Nns for first gal in region 0/86996 for thetabatch 0/48 in 0.0021 seconds for 161 theta-combis
Done region 0/86996 for thetabatch 0/48
Doing region 86/86996 for thetabatch 0/48
Computed Wn for first gal in region 86/86996 for thetabatch 0/48 in 0.0052 seconds
Allocated Nns for first gal in region 86/86996 for thetabatch 0/48 in 0.0019 seconds for 161 theta-combis
Done region 86/86996 for thetabatch 0/48
Doing region 172/86996 for thetabatch 0/48
Computed Wn for first gal in region 172/86996 for thetabatch 0/48 in 0.0054 seconds
Allocated Nns for first gal in region 172/86996 for thetabatch 0/48 in 0.0019 seconds for 161 theta-combis
Done region 172/86996 for thetabatch 0/48
Doing region 258/86996 for thetabatch 0/48
Computed Wn for first gal in region 258/86996 for thetabatch 0/48 in 0.0046 seconds
Allocated Nns for 

In [ ]:
fourpcf.multipoles2npcf

AttributeError: 'NoneType' object has no attribute 'shape'